**Window functions in sql**

| **Category**               | **Functions**                                                     |
| -------------------------- | ----------------------------------------------------------------- |
| Aggregate                  | `SUM`, `AVG`, `MIN`, `MAX`, `COUNT`                               |
| Ranking                    | `ROW_NUMBER`, `RANK`, `DENSE_RANK`, `NTILE`                       |
| Analytic / Value           | `LAG`, `LEAD`, `FIRST_VALUE`, `LAST_VALUE`                        |
| Statistical / Distribution | `PERCENT_RANK`, `CUME_DIST`, `PERCENTILE_CONT`, `PERCENTILE_DISC` |


In [0]:
drop table if exists Employees;

CREATE TABLE Employees (
    EmpID INT PRIMARY KEY,
    EmpName VARCHAR(50),
    Department VARCHAR(50),
    Salary INT,
    HireDate DATE
);


In [0]:
INSERT INTO Employees VALUES
(1, 'Alice',   'HR',        3000, '2020-01-10'),
(2, 'Bob',     'HR',        4000, '2019-03-15'),
(3, 'Charlie', 'Finance',   6000, '2018-07-22'),
(4, 'David',   'Finance',   8000, '2021-05-30'),
(5, 'Eve',     'Finance',   9000, '2017-11-12'),
(6, 'Frank',   'IT',       10000, '2019-12-01'),
(7, 'Grace',   'IT',        7000, '2021-08-19'),
(8, 'Hannah',  'IT',        7500, '2020-10-05'),
(9, 'Ian',     'HR',        5000, '2022-02-25'),
(10,'Jack',    'Finance',   5500, '2021-09-10');


-----> RANKING WINDOW FUNCTIONS

**Q1. Rank employees by Salary (highest first).**

In [0]:
select * from Employees

In [0]:
select
  EmpID,
  EmpName,
  Department,
  Salary,
  row_number() over (order by Salary desc) as row_number,
  rank() over (order by salary desc) as rank,
  dense_rank() over (order by salary desc) as dense_rank
from
  Employees

**Give DENSE_RANK() for employees within each Department.**

In [0]:
select
  empname,
  department,
  salary,
  dense_rank() over (partition by department order by salary desc) as row_number
from
  employees

**Q3. Assign Row Numbers to employees ordered by HireDate.**

In [0]:
SELECT EmpName, HireDate,
 ROW_NUMBER() OVER(ORDER BY HireDate) AS RowNum
FROM Employees;

**Split employees into 4 Quartiles by Salary (NTILE).**

NTILE(N) = splits rows into N roughly equal groups.

Used for quartiles, deciles, percentiles, customer segmentation, sales performance tiers, bonus slabs.

NTILE(4) → Quartiles (25% groups).

NTILE(10) → Deciles (10% groups).

NTILE(100) → Percentiles (1% groups).

**We want to split employees into 4 quartiles based on their salary (highest to lowest).

Quartile 1 (Q1): Top 25% salaries (highest earners)

Quartile 2 (Q2): Next 25%

Quartile 3 (Q3): Next 25%

Quartile 4 (Q4): Lowest 25% salaries**

In [0]:
select EmpName,Salary,
ntile(4) over (order by Salary desc) as  quartiles
from employees

-----> Aggregate window functions

**Q1. Running total of salaries by hire date**

In [0]:
select empname,department,salary,
sum(salary) over(order by salary desc) as running_total
from employees

**Q2. Average salary in each department**

In [0]:
SELECT 
    EmpName, 
    Department, 
    Salary,
    AVG(Salary) OVER(PARTITION BY Department) AS DeptAvgSalary
FROM Employees;

**Q3. Minimum & Maximum salary in each department**

In [0]:
SELECT 
    EmpName, 
    Department, 
    Salary,
    MIN(Salary) OVER(PARTITION BY Department) AS DeptMinSalary,
    MAX(Salary) OVER(PARTITION BY Department) AS DeptMaxSalary
FROM Employees;


**Analytics Window functions**

Lead()
Lag()
Fist_value()
Last_Value()

In [0]:
SELECT 
    EmpName, 
    Salary,
    LAG(Salary) OVER(ORDER BY Salary DESC) AS PrevSalary,
    LEAD(Salary) OVER(ORDER BY Salary DESC) AS NextSalary
FROM Employees;


FIRST_VALUE()

----> Get the highest salary in the partition.

In [0]:
SELECT 
    EmpName, 
    Department,
    Salary,
    FIRST_VALUE(Salary) OVER(PARTITION BY Department ORDER BY Salary DESC) AS DeptTopSalary
FROM Employees;

LAST_VALUE()

Get the lowest salary in the partition.

In [0]:
SELECT 
    EmpName, 
    Department,
    Salary,
    LAST_VALUE(Salary) OVER( ORDER BY Salary DESC 
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS DeptLowestSalary
FROM Employees;

In [0]:
SELECT 
    EmpName, 
    Department,
    Salary,
    LAST_VALUE(Salary) OVER(PARTITION BY Department ORDER BY Salary DESC 
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS DeptLowestSalary
FROM Employees;


**Statistical Window Functions**
1. PERCENT_RANK()

Returns the relative rank of a row between 0 and 1.

Formula:

PERCENT_RANK = Rank − 1 (%) TotalRows − 1


First row = 0, last row = 1.

In [0]:
SELECT 
    EmpName,
    Salary,
    PERCENT_RANK() OVER(ORDER BY Salary) AS PercentRank
FROM Employees;

------ > percentile_cont()


In [0]:
SELECT *,
    PERCENTILE_CONT(0.5) 
        WITHIN GROUP (ORDER BY Salary) 
        OVER() AS MedianSalary
FROM Employees;


percintile_disc -----> same function but it picks the nearest value

In [0]:
SELECT *,
    PERCENTILE_disc(0.5) 
        WITHIN GROUP (ORDER BY Salary) 
        OVER() AS MedianSalary
FROM Employees;